In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
d = pd.read_csv('songz.csv')

In [3]:
d = d.dropna()

In [4]:
newDF = pd.DataFrame(data = {'mood': ['relaxed', 'angry', 'happy', 'sad'], 'mood score': [0,1,2,3]})

In [5]:
newDF

,mood,mood score
0,relaxed,0
1,angry,1
2,happy,2
3,sad,3


In [6]:
## we have mapped 0 to relaxed, 1 to angry, 2 to happy, and 3 to sad

In [7]:
from sklearn.utils import shuffle
X = d.loc[:, ['val','dB','bpm', 'nrgy', 'dnce']]
y = d[['mood']]


In [8]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# le = LabelEncoder()
# X.loc[:,'title'] = le.fit_transform(X.loc[:,'title'])

In [9]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# le2 = LabelEncoder()
# X.loc[:,'artist'] = le2.fit_transform(X.loc[:,'artist'])

In [10]:
X = np.array(X)
X, y = shuffle(X, y)

In [11]:
X_test = X[1200:]
y_test = y[1200:]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)
y_test = scaler.fit_transform(y_test)
# X_test,_ = standardize(X_test, [0])
# y_test, _ = standardize(y_test, [0])
from keras.utils import np_utils
from keras.utils import normalize
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
y_test = np_utils.to_categorical(encoded_Y)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
Using TensorFlow backend.
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
X_test

array([[0.92636858, 0.63893823, 0.48170571, 0.80598487, 0.62132353],
       [0.98826166, 0.74773041, 0.43981796, 0.91340568, 0.7622549 ],
       [0.14950379, 0.53952043, 0.09451241, 0.06390442, 0.41053922],
       ...,
       [0.17084623, 0.80229919, 0.74623516, 0.68431437, 0.33946078],
       [0.17191335, 0.27836984, 0.40950527, 0.03321276, 0.64215686],
       [0.87728097, 0.36495954, 0.45766199, 0.35437904, 0.3872549 ]])

In [13]:
X_train = X[0:900]
Y_train = y[0:900]
X_val = X[900:1200]
Y_val = y[900:1200]

In [14]:

X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
Y_train= scaler.fit_transform(Y_train)
Y_val = scaler.fit_transform(Y_val)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [15]:
from keras.utils import np_utils
from keras.utils import normalize
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y = encoder.transform(Y_train)

Y_train =  np_utils.to_categorical(encoded_Y)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
from keras.utils import np_utils
from keras.utils import normalize
encoder = LabelEncoder()
encoder.fit(Y_val)
encoded_Y = encoder.transform(Y_val)
Y_val = np_utils.to_categorical(encoded_Y)

In [17]:
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras import regularizers
from keras.optimizers import Adam, SGD

model = Sequential()

# model.add(Dense(64, input_shape=(7,), activation='relu'))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.001))
# model.add(Dropout(0.4))
model.add(Dense(48, activation='relu', input_shape=(5,)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(24))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Dense(12))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Dense(6))
# model.add(BatchNormalization())
# model.add(Dropout(0.4))

# model.add(Dense(64, activation='relu'))
# #model.add(BatchNormalization())
# model.add(Dropout(0.2))


model.add(Dense(4, activation='sigmoid', name='output'))


model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 48)                288       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48)                192       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48)                0         
_________________________________________________________________
output (Dense)               (None, 4)                 196       
Total params: 676
Trainable params: 580
Non-trainable params: 96
_________________________________________________________________


In [18]:
# Adam optimizer with learning rate of 0.001
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
optimizer = Adam(lr=0.001)
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-10, decay=0)
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer = 'adam' , loss='binary_crossentropy', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
hist = model.fit(X_train, Y_train, verbose=2, batch_size=128, epochs=50, validation_data=(X_val,Y_val),callbacks=[mc])

Instructions for updating:
Use tf.cast instead.
Train on 900 samples, validate on 300 samples
Epoch 1/50
 - 1s - loss: 0.8456 - acc: 0.4969 - val_loss: 0.7592 - val_acc: 0.5333

Epoch 00001: val_acc improved from -inf to 0.53333, saving model to best_model.h5
Epoch 2/50
 - 0s - loss: 0.8006 - acc: 0.5011 - val_loss: 0.7041 - val_acc: 0.5858

Epoch 00002: val_acc improved from 0.53333 to 0.58583, saving model to best_model.h5
Epoch 3/50
 - 0s - loss: 0.7652 - acc: 0.5283 - val_loss: 0.6665 - val_acc: 0.6250

Epoch 00003: val_acc improved from 0.58583 to 0.62500, saving model to best_model.h5
Epoch 4/50
 - 0s - loss: 0.7343 - acc: 0.5317 - val_loss: 0.6448 - val_acc: 0.6517

Epoch 00004: val_acc improved from 0.62500 to 0.65167, saving model to best_model.h5
Epoch 5/50
 - 0s - loss: 0.7095 - acc: 0.5631 - val_loss: 0.6335 - val_acc: 0.6567

Epoch 00005: val_acc improved from 0.65167 to 0.65667, saving model to best_model.h5
Epoch 6/50
 - 0s - loss: 0.6894 - acc: 0.5769 - val_loss: 0.6272

In [19]:
scores = model.evaluate(X_test, y_test, verbose=0)

In [20]:
scores

[0.5361460664414412, 0.7508278145695364]

In [21]:
import numpy as np
import matplotlib.pyplot as plt
plt.figure(1, figsize=(10,7))

# Training accuracy
plt.subplot(221)
plt.plot(range(0,50), hist.history['acc'], c = 'forestgreen');
plt.title('Training accuracy per epoch');

# Training loss #
plt.subplot(222)
plt.plot(range(0,50), hist.history['loss'], c = 'forestgreen');
plt.title('Training loss per epoch');

# Validation accuracy #
plt.subplot(223)
plt.plot(range(0,50), hist.history['val_acc'], c = 'blue');
plt.title('Validation accuracy per epoch');

# Validation loss #
plt.subplot(224)
plt.plot(range(0,50), hist.history['val_loss'], c = 'blue');
plt.title('Validation loss per epoch');

In [22]:
import pickle
filename = 'song_NN.pickle'

pickle_out = open(filename, 'wb')

pickle.dump(model, pickle_out)
pickle_out.close()

In [23]:
pickle_in = open("song_NN.pickle","rb")
example_dict = pickle.load(pickle_in)
y_pred = example_dict.predict(X_test)

In [24]:
X_test

array([[0.92636858, 0.63893823, 0.48170571, 0.80598487, 0.62132353],
       [0.98826166, 0.74773041, 0.43981796, 0.91340568, 0.7622549 ],
       [0.14950379, 0.53952043, 0.09451241, 0.06390442, 0.41053922],
       ...,
       [0.17084623, 0.80229919, 0.74623516, 0.68431437, 0.33946078],
       [0.17191335, 0.27836984, 0.40950527, 0.03321276, 0.64215686],
       [0.87728097, 0.36495954, 0.45766199, 0.35437904, 0.3872549 ]])

In [25]:
y_pred

array([[0.21431434, 0.20859206, 0.31698182, 0.2460368 ],
       [0.18490916, 0.20987427, 0.31353402, 0.30317605],
       [0.38156712, 0.02891389, 0.688908  , 0.2196776 ],
       ...,
       [0.16889727, 0.33741376, 0.25874868, 0.27489048],
       [0.09442896, 0.03617036, 0.29641145, 0.24646467],
       [0.37267974, 0.0679768 , 0.40493056, 0.13001296]], dtype=float32)

In [26]:
np.argmax(y_pred,axis = 1)

array([2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 1, 0, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 0, 2, 0, 2, 2, 2, 1,
       2, 2, 3, 1, 2, 2, 0, 0, 2, 1, 2, 1, 2, 2, 2, 0, 1, 2, 1, 2, 1, 0,
       2, 0, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 3, 3, 1, 2, 2, 0, 3,
       0, 3, 2, 2, 2, 2, 2, 3, 1, 1, 1, 2, 2, 2, 0, 2, 2, 1, 2, 0, 2, 2,
       2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 1, 2, 2, 2, 1, 3, 2, 3, 2, 2, 1, 2,
       3, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 0, 2, 3, 3, 3, 2, 2, 1, 3, 1, 2,
       2, 1, 2, 2, 2, 2, 1, 2, 2, 3, 3, 2, 2, 1, 1, 2, 1, 2, 3, 1, 2, 2,
       0, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 3, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 1, 3, 3, 2, 3, 2, 3, 1, 0, 2, 3, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2,
       2, 3, 1, 1, 1, 1, 2, 3, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0,
       2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 3, 2, 2, 0, 1,
       1, 1, 1, 2, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 1, 2, 0,
       1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 0, 2, 1, 2,

In [27]:
acc = []
for i,j in zip(np.argmax(y_test,axis = 1),np.argmax(y_pred,axis = 1)):
    if i == j:
        acc.append(1)

In [28]:
np.argmax(y_test,axis = 1)

array([1, 1, 2, 0, 2, 2, 3, 2, 2, 1, 2, 1, 2, 1, 0, 1, 1, 1, 0, 2, 2, 2,
       0, 0, 2, 2, 0, 3, 0, 0, 1, 3, 2, 2, 3, 1, 1, 3, 0, 3, 2, 1, 3, 1,
       3, 2, 3, 1, 0, 0, 0, 0, 2, 1, 0, 3, 0, 0, 1, 0, 0, 2, 2, 1, 1, 1,
       3, 0, 2, 2, 2, 1, 2, 2, 2, 3, 0, 0, 2, 2, 3, 3, 3, 2, 2, 3, 0, 1,
       2, 3, 2, 2, 2, 0, 1, 0, 1, 2, 1, 1, 3, 1, 3, 0, 0, 0, 0, 0, 1, 0,
       3, 3, 3, 2, 2, 1, 1, 2, 2, 2, 1, 0, 1, 0, 3, 2, 3, 2, 0, 2, 1, 1,
       2, 1, 0, 1, 1, 1, 0, 1, 2, 2, 0, 0, 2, 2, 2, 3, 3, 0, 2, 1, 1, 0,
       0, 1, 0, 2, 2, 3, 0, 2, 2, 2, 1, 0, 1, 3, 0, 1, 2, 2, 3, 1, 3, 2,
       1, 2, 2, 2, 1, 2, 2, 1, 1, 3, 1, 3, 0, 0, 2, 2, 2, 2, 0, 1, 2, 2,
       2, 1, 0, 3, 2, 2, 0, 3, 1, 1, 2, 1, 3, 0, 3, 3, 3, 1, 2, 0, 2, 0,
       1, 0, 0, 2, 3, 0, 1, 3, 2, 1, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2,
       0, 3, 1, 3, 3, 0, 1, 1, 1, 2, 3, 2, 3, 2, 2, 0, 2, 3, 2, 2, 3, 1,
       2, 1, 2, 0, 3, 2, 2, 3, 3, 2, 2, 1, 1, 0, 2, 3, 3, 3, 2, 0, 1, 3,
       2, 3, 1, 3, 2, 3, 1, 0, 1, 2, 2, 0, 0, 2, 2,

In [29]:
len(acc)/len(np.argmax(y_test,axis = 1))

0.423841059602649